In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds


In [2]:
mnist_dataset,mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)

# Preprocessing

In [3]:
#train test split
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']
mnist_train


<_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

In [4]:
#validation set
num_validation_samples=0.1*mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)

num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)


In [5]:
mnist_info.splits['train'].num_examples

60000

In [6]:
num_test_samples

<tf.Tensor: shape=(), dtype=int64, numpy=10000>

In [7]:
#scaling mnist matrix data from 0-255 to 0-1
def scale(image,label):
    image=tf.cast(image,tf.float32)
    image=image/255.
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)


In [8]:
scaled_train_and_validation_data

<MapDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [9]:
#scale test data
test_data=mnist_test.map(scale)
test_data

<MapDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [10]:
#shuffle data before spliting
buffer_size=10000
shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(buffer_size)

In [11]:
#split train and validation datasets
validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

In [12]:
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)


validation_inputs, validation_targets = next(iter(validation_data))
test_data


<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

# Outline the model

In [13]:
input_size = 784
output_size = 10

hidden_layer_size = 50

model = tf.keras.Sequential([
    

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer

    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

no_of_epochs=5
model.fit(train_data,epochs=no_of_epochs,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/5
540/540 - 6s - loss: 0.4082 - accuracy: 0.8844 - val_loss: 0.2230 - val_accuracy: 0.9397
Epoch 2/5
540/540 - 6s - loss: 0.1845 - accuracy: 0.9462 - val_loss: 0.1835 - val_accuracy: 0.9502
Epoch 3/5
540/540 - 6s - loss: 0.1408 - accuracy: 0.9579 - val_loss: 0.1462 - val_accuracy: 0.9608
Epoch 4/5
540/540 - 6s - loss: 0.1155 - accuracy: 0.9664 - val_loss: 0.1134 - val_accuracy: 0.9662
Epoch 5/5
540/540 - 6s - loss: 0.0977 - accuracy: 0.9706 - val_loss: 0.1078 - val_accuracy: 0.9677


In [14]:
model.evaluate(test_data)

1/1 [==============================] - 0s 32ms/step - loss: 0.1222 - accuracy: 0.9622


[0.12220898270606995, 0.9621999859809875]

In [15]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 [==============================] - 0s 33ms/step - loss: 0.1222 - accuracy: 0.9622
Test loss: 0.12. Test accuracy: 96.22%
